# Qwen-Image-Edit 图像编辑演示

本笔记本演示如何使用 Qwen-Image-Edit 模型进行各种图像编辑任务，包括：
- 语义编辑（风格转换、视角变换等）
- 外观编辑（添加/删除对象、修改颜色等）
- 文本编辑（修改图像中的文字）
- 链式编辑（逐步修正错误）

In [ ]:
# 安装必要的依赖
!pip install modelscope ipywidgets tqdm accelerate diffusers transformers

In [ ]:
# 检查版本信息
from importlib.metadata import version
print("diffusers version:", version("diffusers"))
print("torch version:", version("torch"))
print("transformers version:", version("transformers"))
print("PIL version:", version("Pillow"))

In [ ]:
# 下载 Qwen-Image-Edit 模型
from modelscope import snapshot_download
import os

def download_edit_model(model_id="Qwen/Qwen-Image-Edit", local_dir='./models/Qwen-Image-Edit'):
    """
    下载图像编辑模型
    """
    print(f"开始下载 {model_id} 模型到: {local_dir}")
    
    # 确保目录存在
    os.makedirs(os.path.dirname(local_dir), exist_ok=True)
    
    # 下载模型
    snapshot_download(
        model_id, 
        local_dir=local_dir
    )
    
    print(f"模型下载完成，保存在: {local_dir}")

# 下载模型
download_edit_model()

In [ ]:
# 设置设备和数据类型
import torch
from PIL import Image
import os

# 检测设备
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.bfloat16
    print(f"使用 GPU: {torch.cuda.get_device_name()}")
    print(f"GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = "cpu"
    torch_dtype = torch.float32
    print("使用 CPU")

print(f"设备: {device}, 数据类型: {torch_dtype}")

In [ ]:
# 加载图像编辑管道
from diffusers import QwenImageEditPipeline

model_path = "./models/Qwen-Image-Edit"

print("正在加载 Qwen-Image-Edit 管道...")
pipeline = QwenImageEditPipeline.from_pretrained(model_path, torch_dtype=torch_dtype,use_safetensors=True,device_map="balanced")
print("管道加载完成")

# 设置数据类型和设备
# pipeline.to(torch_dtype)
# pipeline.to(device)
# pipeline.set_progress_bar_config(disable=None)

print(f"管道已配置到 {device} 设备，数据类型: {torch_dtype}")

In [ ]:
# 辅助函数
def edit_image(image_path, prompt, output_path=None, seed=42, steps=50, cfg_scale=4.0):
    """
    编辑图像的通用函数
    
    Args:
        image_path: 输入图像路径
        prompt: 编辑提示词
        output_path: 输出图像路径（可选）
        seed: 随机种子
        steps: 推理步数
        cfg_scale: CFG 缩放因子
    
    Returns:
        PIL Image: 编辑后的图像
    """
    # 加载输入图像
    if isinstance(image_path, str):
        image = Image.open(image_path).convert("RGB")
    else:
        image = image_path
    
    print(f"正在编辑图像...")
    print(f"提示词: {prompt}")
    
    # 编辑参数
    inputs = {
        "image": image,
        "prompt": prompt,
        "generator": torch.manual_seed(seed),
        "true_cfg_scale": cfg_scale,
        "negative_prompt": " ",
        "num_inference_steps": steps,
    }
    
    # 执行编辑
    with torch.inference_mode():
        output = pipeline(**inputs)
        edited_image = output.images[0]
    
    # 保存图像
    if output_path:
        edited_image.save(output_path)
        print(f"编辑后的图像已保存到: {os.path.abspath(output_path)}")
    
    return edited_image

def display_before_after(original_path, edited_image, title="图像编辑对比"):
    """
    显示编辑前后的对比图
    """
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # 原图
    if isinstance(original_path, str):
        original = Image.open(original_path)
    else:
        original = original_path
    axes[0].imshow(original)
    axes[0].set_title("原图")
    axes[0].axis('off')
    
    # 编辑后
    axes[1].imshow(edited_image)
    axes[1].set_title("编辑后")
    axes[1].axis('off')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

print("辅助函数已定义完成")

## 演示案例

以下是各种图像编辑任务的演示。请确保在 `./demo_images/` 目录下准备好相应的测试图像。

In [ ]:
# 创建演示目录
demo_dir = "./demo_images"
output_dir = "./edited_images"

os.makedirs(demo_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(f"演示目录已创建: {demo_dir}")
print(f"输出目录已创建: {output_dir}")
print("\n请将测试图像放入 demo_images 目录中")

### 1. 语义编辑演示

In [ ]:
# 风格转换示例
input_image = "./demo_images/portrait.jpg"  # 请准备一张人像图片

if os.path.exists(input_image):
    # 转换为宫崎骏风格
    style_prompts = [
        "Transform this portrait into Studio Ghibli animation style, with soft colors and dreamy atmosphere",
        "Convert to oil painting style with impressionist brushstrokes",
        "Transform into cyberpunk style with neon lighting and futuristic elements"
    ]
    
    for i, prompt in enumerate(style_prompts):
        output_path = f"{output_dir}/style_transfer_{i+1}.png"
        edited = edit_image(input_image, prompt, output_path, seed=42+i)
        display_before_after(input_image, edited, f"风格转换 {i+1}: {prompt[:30]}...")
else:
    print(f"请将人像图片保存为: {input_image}")

In [ ]:
# 视角变换示例
input_image = "./demo_images/object.jpg"  # 请准备一张物体图片

if os.path.exists(input_image):
    viewpoint_prompts = [
        "Rotate this object 90 degrees to show the side view",
        "Show the back view of this object by rotating it 180 degrees",
        "Change the viewing angle to show this object from above"
    ]
    
    for i, prompt in enumerate(viewpoint_prompts):
        output_path = f"{output_dir}/viewpoint_{i+1}.png"
        edited = edit_image(input_image, prompt, output_path, seed=100+i)
        display_before_after(input_image, edited, f"视角变换 {i+1}")
else:
    print(f"请将物体图片保存为: {input_image}")

### 2. 外观编辑演示

In [ ]:
# 添加对象示例
input_image = "./demo_images/scene.jpg"  # 请准备一张场景图片

if os.path.exists(input_image):
    addition_prompts = [
        "Add a red signboard with text 'OPEN' to this scene, include realistic reflection",
        "Add a cute cat sitting in the foreground",
        "Add some colorful balloons floating in the sky"
    ]
    
    for i, prompt in enumerate(addition_prompts):
        output_path = f"{output_dir}/addition_{i+1}.png"
        edited = edit_image(input_image, prompt, output_path, seed=200+i)
        display_before_after(input_image, edited, f"添加对象 {i+1}")
else:
    print(f"请将场景图片保存为: {input_image}")

In [ ]:
# 颜色修改示例
input_image = "./demo_images/colorful_object.jpg"  # 请准备一张彩色物体图片

if os.path.exists(input_image):
    color_prompts = [
        "Change the main object's color to bright purple",
        "Make this image black and white except for one red element",
        "Change the background to a warm sunset color palette"
    ]
    
    for i, prompt in enumerate(color_prompts):
        output_path = f"{output_dir}/color_change_{i+1}.png"
        edited = edit_image(input_image, prompt, output_path, seed=400+i)
        display_before_after(input_image, edited, f"颜色修改 {i+1}")
else:
    print(f"请将彩色物体图片保存为: {input_image}")

### 3. 文本编辑演示

In [ ]:
# 文本修改示例
input_image = "./demo_images/text_image.jpg"  # 请准备一张包含文字的图片

if os.path.exists(input_image):
    text_prompts = [
        "Change the text in this image to 'HELLO WORLD' in the same style",
        "Replace any English text with Chinese characters '欢迎光临'",
        "Change the color of the text to bright blue while keeping everything else the same"
    ]
    
    for i, prompt in enumerate(text_prompts):
        output_path = f"{output_dir}/text_edit_{i+1}.png"
        edited = edit_image(input_image, prompt, output_path, seed=500+i)
        display_before_after(input_image, edited, f"文本编辑 {i+1}")
else:
    print(f"请将包含文字的图片保存为: {input_image}")

### 4. 链式编辑演示

In [ ]:
# 链式编辑示例 - 逐步完善图像
input_image = "./demo_images/base_image.jpg"  # 请准备一张基础图片

if os.path.exists(input_image):
    # 第一步编辑
    step1_prompt = "Add a beautiful garden with flowers in the background"
    step1_result = edit_image(input_image, step1_prompt, f"{output_dir}/chain_step1.png", seed=600)
    display_before_after(input_image, step1_result, "链式编辑 - 步骤1: 添加花园")
    
    # 第二步编辑（基于第一步结果）
    step2_prompt = "Add a wooden bench in the garden for people to sit"
    step2_result = edit_image(step1_result, step2_prompt, f"{output_dir}/chain_step2.png", seed=601)
    display_before_after(step1_result, step2_result, "链式编辑 - 步骤2: 添加长椅")
    
    # 第三步编辑（基于第二步结果）
    step3_prompt = "Add warm golden hour lighting to create a romantic atmosphere"
    step3_result = edit_image(step2_result, step3_prompt, f"{output_dir}/chain_step3.png", seed=602)
    display_before_after(step2_result, step3_result, "链式编辑 - 步骤3: 添加黄金时光照明")
    
    # 显示最终对比
    display_before_after(input_image, step3_result, "链式编辑 - 最终对比")
    
else:
    print(f"请将基础图片保存为: {input_image}")

### 5. 批量处理演示

In [ ]:
# 批量处理示例
import glob

# 获取所有演示图片
demo_images = glob.glob(f"{demo_dir}/*.jpg") + glob.glob(f"{demo_dir}/*.png")

if demo_images:
    batch_prompt = "Enhance this image with better lighting and more vibrant colors"
    
    print(f"找到 {len(demo_images)} 张图片进行批量处理")
    
    for i, img_path in enumerate(demo_images[:3]):  # 限制处理前3张图片
        print(f"\n处理第 {i+1} 张图片: {os.path.basename(img_path)}")
        
        output_path = f"{output_dir}/batch_enhanced_{i+1}.png"
        edited = edit_image(img_path, batch_prompt, output_path, seed=700+i)
        
        # 显示对比
        display_before_after(img_path, edited, f"批量增强 {i+1}")
        
    print("\n批量处理完成！")
else:
    print("未找到演示图片，请在 demo_images 目录中添加图片")

## 总结

本笔记本演示了 Qwen-Image-Edit 模型的强大图像编辑能力：

1. **语义编辑**: 风格转换、视角变换等高级编辑
2. **外观编辑**: 添加/删除对象、修改颜色等精确编辑
3. **文本编辑**: 修改图像中的文字内容和样式
4. **链式编辑**: 通过多步编辑实现复杂的图像变换
5. **批量处理**: 高效处理多张图像

### 使用建议：
- 使用清晰、具体的提示词获得更好的编辑效果
- 对于复杂编辑，可以使用链式编辑方法逐步完善
- 调整 `cfg_scale` 和 `num_inference_steps` 参数来平衡质量和速度
- 使用不同的随机种子获得多样化的编辑结果